
**🌾 Project Kisan – AI-Powered Crop Assistance System**
> 
Smart Guidance • Farmer Journal • Gov Scheme Info • Expert Recommendation

📌 Overview

Project Kisan is an AI-powered agricultural support system designed to help farmers with real-time crop guidance, expert recommendations, government scheme info, and daily farm logging.
This notebook integrates multiple AI agents to deliver fast, safe, and beginner-friendly crop support—without making medical-grade decisions or pesticide dosage recommendations.


**🚀 Key Features
1️⃣ Crop Issue Analysis (Safe, Non-Diagnostic)**


Uses CropDiagnosisAgent + CropCareAgent to provide:

✔ General reason for crop stress

✔ Risk level (Low / Medium / High)

✔ Red-flag symptoms

✔ Safe first-aid crop steps

❌ No harmful suggestions, no chemicals

**2️⃣ Daily Farm Log System**

Your DailyFarmLogAgent can store and retrieve:

Date & time

Crop name

Symptoms observed

Field condition

Watering details

Notes from the farmer

All logs can be fetched using:

memory_bank.get_logs()

**3️⃣ Smart Crop Care Guidance**

Delivered by CropCareAgent:

Irrigation schedule

Sunlight requirements

Soil moisture guidance

Organic, natural remedies

What to avoid (overwatering, excess fertilizer)

**4️⃣ Agriculture Expert Recommendation**

Provided by FarmExpertAgent, suggesting the right expert:

Soil Expert

Irrigation Specialist

Plant Pathologist

Pest Management Officer

KVK Officer

Agriculture Extension Officer

**5️⃣ Government Schemes & Farmer FAQs**


Handled by KrishiFAQAgent:

PM-Kisan

PMFBY Crop Insurance

Soil Health Card

Fertilizer Subsidies

Organic Farming Info

Crops & Varieties (Simple Wikipedia summary)

**6️⃣ Parallel AI Processing (Fast Output)**

Function:

parallel_crop_analysis()


Returns:

Crop Care Guidance

Expert Recommendation
At the same time (async parallel tasks)

**7️⃣ General Crop Information**

Auto-fetched Wikipedia summary:

What the crop is

Where it grows

Common uses

Simple background

**8️⃣ Farmer Memory / Journal**

Your MemoryBank keeps:

All past issues

Logs

Patterns

Crop history

Field health timeline

Used for improved guidance over time.

**9️⃣ Final Orchestrator Output**

When you run:

output = await orch.run(crop_name, issue)


You receive:
| Output Key                         | Meaning                 |
| ---------------------------------- | ----------------------- |
| **crop_care_guidance**             | Full AI crop care steps |
| **recommended_agriculture_expert** | Best expert to meet     |
| **general_crop_information**       | Clean summary of crop   |
| **farmer_journal**                 | Saved logs / memory     |


**Output Key	Meaning**
crop_care_guidance	Full AI crop care steps
recommended_agriculture_expert	Best expert to meet
general_crop_information	Clean summary of crop
farmer_journal	Saved logs / memory

**🧪 Example Output Format**
🌾 Crop Care Summary:
Your plant may be experiencing mild water stress. Ensure proper soil moisture.

👨‍🌾 Recommended Agriculture Expert:
Plant Pathologist

📘 General Crop Information:
Tomato is a widely grown vegetable crop known for its nutritional value…

📝 Farmer Journal:
- 2025-11-25: Tomato field had yellow leaves and low soil moisture.

**** Conclusion****

Project Kisan is a complete AI-powered agriculture help system with:

Multi-agent intelligence

Safe guidance

Government scheme awareness

Farmer history tracking

Expert recommendation

Rapid parallel analysis

In [1]:
# CELL 1 — INSTALLS (run once; Kaggle may already have many packages)
!pip install --upgrade pip
!pip install google-generativeai wikipedia aiohttp nest_asyncio pillow geopy pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=682513af87c234634481982db4b4a255d6328544f18d8a614071d2bb1b0fb4e1
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
   ━━━━━━━━━━━━━━━━━━━

In [2]:
# CELL 2 — IMPORTS & JUPYTER ASYNC FIX
import os
import json
import asyncio
import nest_asyncio
from datetime import datetime

import google.generativeai as genai
import wikipedia

# helper libs
import aiohttp
from geopy.geocoders import Nominatim
from PIL import Image
import pandas as pd
import numpy as np

nest_asyncio.apply()


In [3]:
# CELL 3 — KAGGLE SECRET (set GOOGLE_API_KEY in Kaggle notebook Secrets)
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ GOOGLE_API_KEY loaded from Kaggle secrets.")
except Exception as e:
    print("🔑 Could not load GOOGLE_API_KEY from Kaggle secrets:", e)
    print("→ Add the key to Notebook Settings > Secrets and re-run this cell.")


✅ GOOGLE_API_KEY loaded from Kaggle secrets.


In [4]:
# CELL 4 — GEMINI / GENERATIVE MODEL INIT
api_key = os.environ.get("GOOGLE_API_KEY", None)
if api_key is None:
    print("⚠️ GOOGLE_API_KEY not found. Model calls will be disabled.")
    model = None
else:
    genai.configure(api_key=api_key)
    # adjust model name if needed by API
    model = genai.GenerativeModel("gemini-2.5-pro")
    print("✅ Gemini model configured (gemini-2.5-pro).")


✅ Gemini model configured (gemini-2.5-pro).


In [5]:
# CELL 5 — MEMORY BANK (Farm journal)
class MemoryBank:
    def __init__(self, path="farm_journal.json"):
        self.journal_path = path
        try:
            with open(self.journal_path, "r") as f:
                self.data = json.load(f)
        except:
            self.data = {"logs": []}

    def add_log(self, entry):
        self.data["logs"].append(entry)
        with open(self.journal_path, "w") as f:
            json.dump(self.data, f, indent=2)

    def get_logs(self):
        return self.data["logs"]

memory_bank = MemoryBank()
print("✅ MemoryBank initialized:", memory_bank.journal_path)


✅ MemoryBank initialized: farm_journal.json


In [6]:
# CELL 6 — AGENTS (CropDiagnosis / CropCare / FAQ / Expert / DailyLog)
# Note: model.generate_content usage assumes google-generativeai client supports it.
# If your installed client uses a different call, adapt model invocation accordingly.

class CropDiagnosisAgent:
    async def analyze(self, crop_name, symptoms):
        prompt = f"""
Crop: {crop_name}

Observed Signs: {symptoms}

Provide ONLY general, non-diagnostic guidance:
- Possible non-specific causes (nutrient stress, water stress, pests)
- Risk category (low / moderate / high)
- Red-flag signs (seek local expert)
- Practical first steps (watering, inspect, isolate plants)
- Use farmer-friendly language; avoid disease names.
"""
        if model is None:
            return "[LLM not configured] — cannot generate guidance."
        res = model.generate_content(prompt)
        return getattr(res, "text", str(res))


class CropCareAgent:
    async def suggest(self, crop_name, visible_issue):
        prompt = f"""
Crop: {crop_name}

Visible Issue Observed: {visible_issue}

Provide ONLY general farming guidance:
- Watering & irrigation guidance
- General nutrient / organic suggestions (non-prescriptive)
- Things to avoid (overwatering, pesticide misuse)
- Simple care steps farmer can follow
"""
        if model is None:
            return "[LLM not configured] — cannot generate suggestion."
        res = model.generate_content(prompt)
        return getattr(res, "text", str(res))


class DailyFarmLogAgent:
    def log_day(self, crop_name, field_condition, issues, water_level):
        entry = {
            "date": str(datetime.now()),
            "crop": crop_name,
            "field_condition": field_condition,
            "issues_observed": issues,
            "water_level": water_level
        }
        memory_bank.add_log(entry)
        return "🌾 Daily farm log saved successfully."


class KrishiFAQAgent:
    async def search_faq(self, query):
        try:
            page = wikipedia.summary(query + " agriculture", sentences=3)
            return page
        except Exception:
            try:
                page = wikipedia.summary(query, sentences=3)
                return page
            except Exception:
                return "❗ No general farming information found. Try a different query."


class FarmExpertAgent:
    async def suggest_expert(self, issue):
        prompt = f"""
Based on the farm issue: {issue}

Suggest ONLY the MOST RELEVANT AGRICULTURE EXPERT or SERVICE:
(e.g., Soil Expert, Fertilizer Officer, Plant Pathologist, Irrigation Expert,
 Pest Management Officer, Krishi Vigyan Kendra, Agriculture Extension Officer)

Do NOT diagnose any plant disease. Return only the expert/department name(s).
"""
        if model is None:
            return "[LLM not configured] — cannot suggest expert."
        res = model.generate_content(prompt)
        return getattr(res, "text", str(res))


In [7]:
# CELL 7 — PARALLEL ANALYSIS FUNCTION
async def parallel_crop_analysis(crop_issue):
    care_agent = CropCareAgent()
    expert_agent = FarmExpertAgent()

    care_task = asyncio.create_task(care_agent.suggest(
        crop_name=crop_issue.get("crop"),
        visible_issue=crop_issue.get("issue")
    ))

    expert_task = asyncio.create_task(expert_agent.suggest_expert(
        crop_issue.get("issue")
    ))

    care_response, expert_response = await asyncio.gather(care_task, expert_task)
    return care_response, expert_response


In [8]:
# CELL 8 — ORCHESTRATOR (ties everything together)
class KisanOrchestrator:
    def __init__(self):
        self.log_agent = DailyFarmLogAgent()
        self.care_agent = CropCareAgent()
        self.faq_agent = KrishiFAQAgent()
        self.expert_agent = FarmExpertAgent()

    async def run(self, crop_name, issue):
        # Save event
        memory_bank.add_log({
            "date": str(datetime.now()),
            "event": "crop_issue_check",
            "crop": crop_name,
            "issue": issue
        })

        # Parallel agents
        crop_issue = {"crop": crop_name, "issue": issue}
        care_advice, expert_suggestion = await parallel_crop_analysis(crop_issue)

        # FAQ
        faq_response = await self.faq_agent.search_faq(crop_name)

        return {
            "crop_care_guidance": care_advice,
            "recommended_agriculture_expert": expert_suggestion,
            "general_crop_information": faq_response,
            "farmer_journal": memory_bank.get_logs()
        }

# Create orchestrator instance (use this name in demo)
orch = KisanOrchestrator()
print("✅ Orchestrator initialized.")


✅ Orchestrator initialized.


In [9]:
# CELL 9 — DEMO RUN (execute to test)
# You can change crop_name / issue to test different scenarios.

async def demo_run():
    crop_name = "Tomato"
    issue = "Leaves turning yellow with brown spots"
    output = await orch.run(crop_name, issue)

    print("🌾 Crop Care Summary:")
    print(output["crop_care_guidance"].split("\n")[0])

    print("\n👨‍🌾 Recommended Agriculture Expert:")
    print(output["recommended_agriculture_expert"].strip())

    print("\n📘 General Crop Information:")
    print(output["general_crop_information"].split("\n")[0])

# Run demo in notebook-friendly way
loop = asyncio.get_event_loop()
if loop.is_running():
    await demo_run()            # works because we applied nest_asyncio
else:
    loop.run_until_complete(demo_run())


🌾 Crop Care Summary:
### **Watering & Irrigation Guidance**

👨‍🌾 Recommended Agriculture Expert:
Plant Pathologist
Krishi Vigyan Kendra
Agriculture Extension Officer

📘 General Crop Information:
There are more than 10,000 tomato varieties available.
